MCP Resources

Adding a resource to the server

In [ ]:
from pathlib import Path

In [ ]:
@mcp.resource("file://suggested_titles")
def suggested_titles() -> list[str]:
    """
    Read and return suggested Wikipedia topics from a local file.
    """
    try:
        path = Path("suggested_titles.txt")
        if not path.exists():
            return ["File not found"]
        return path.read_text(encoding="utf-8").strip().splitlines()
    except Exception as e:
        return [f"Error reading file: {str(e)}"]

Listing available resources

In [ ]:
async def list_resources(session):
    try:
        response = await session.list_resources()
        if not response or not response.resources:
            print("No resources found on the server.")
            return

        print("\nAvailable Resources:")
        for i, r in enumerate(response.resources, 1):
            print(f"[{i}] {r.name}")
        print("\nUse: /resource <name> to view its content.")
    except Exception as e:
        print("Failed to list resources:", e)

Handling resource fetch requests

Implement the logic to retrieve and display the content of a specific resource using its name or number

In [ ]:
async def handle_resource(session, command):
    parts = shlex.split(command.strip())
    if len(parts) < 2:
        print("Usage: /resource <name>")
        return

    resource_id = parts[1]

    try:
        # Get all available resources
        response = await session.list_resources()
        resources = response.resources
        resource_map = {str(i + 1): r.name for i, r in enumerate(resources)}

        # Resolve name or index
        resource_name = resource_map.get(resource_id, resource_id)
        match = next((r for r in resources if r.name == resource_name), None)

        if not match:
            print(f"Resource '{resource_id}' not found.")
            return

        # Fetch resource content
        result = await session.read_resource(match.uri)

        for content in result.contents:
            if hasattr(content, "text"):
                print("\n=== Resource Text ===")
                print(content.text)

    except Exception as e:
        print("Resource fetch failed:", e)

Updating the main loop

In [ ]:
async def main():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await load_mcp_tools(session)
            agent = await create_graph(session)

            print("Wikipedia MCP agent is ready.")
            print("Type a question or use the following templates:")
            print("  /prompts                - to list available prompts")
            print("  /prompt <name> \"args\".  - to run a specific prompt")
            print("  /resources              - to list available resources")
            print("  /resource <name>        - to run a specific resource")

            while True:
                user_input = input("\nYou: ").strip()
                if user_input.lower() in {"exit", "quit", "q"}:
                    break
                elif user_input.startswith("/prompts"):
                    await list_prompts(session)
                    continue
                elif user_input.startswith("/prompt"):
                    await handle_prompt(session, tools, user_input, agent)
                    continue
                elif user_input.startswith("/resources"):
                    await list_resources(session)
                    continue
                elif user_input.startswith("/resource"):
                    await handle_resource(session, user_input)
                    continue

                try:
                    response = await agent.ainvoke(
                        {"messages": user_input},
                        config={"configurable": {"thread_id": "wiki-session"}}
                    )
                    print("AI:", response["messages"][-1].content)
                except Exception as e:
                    print("Error:", e)